In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import * 

In [3]:
import numpy as np

In [4]:
coef = np.loadtxt('lab7_ex2_lp_ip_coef.txt', delimiter=',')

In [5]:
print(coef.shape)
print('Num rows:',coef.shape[0])
print('Num cols:',coef.shape[1])

(6, 501)
Num rows: 6
Num cols: 501


In [6]:
print(coef)

[[8.01000e+02 9.56000e+02 5.12000e+02 ... 1.07800e+03 9.48000e+02
  0.00000e+00]
 [7.02000e+02 8.40000e+02 5.13000e+02 ... 7.07000e+02 6.25000e+02
  1.73725e+05]
 [2.46000e+02 9.94000e+02 9.38000e+02 ... 4.23000e+02 6.91000e+02
  1.90019e+05]
 [7.99000e+02 2.28000e+02 1.00000e+01 ... 2.31000e+02 4.40000e+02
  1.86926e+05]
 [4.63000e+02 6.54000e+02 1.67000e+02 ... 7.11000e+02 6.78000e+02
  1.92737e+05]
 [2.52000e+02 7.35000e+02 5.86000e+02 ... 9.47000e+02 7.98000e+02
  1.83645e+05]]


In [7]:
# create a model
model_lab7_ex2 = ConcreteModel()

In [8]:
# we can set certain values using python variables
N = coef.shape[1]-1 #we declare a Python variable N denoting the number of decision variables  
#Note: We subtract a one since the last column corresponds to the scalar (or constraint RHS)

In [9]:
M = coef.shape[0]-1 #we declare a Python variable M denoting the number of constraints
#Note: We subtract a one since the first row corresponds to the objective function. 

In [10]:
# coefficients for objective function can be obtained by accessing the coef array
obj_coef = coef[0,:-1]
print(obj_coef.shape)
print(obj_coef)

(500,)
[ 801.  956.  512.  823.  740.  752.  936.  536.  851.  399.  515.  660.
  523.  229.  637.  609.  663.  561.  516.  709. 1060.  405.  564.  692.
  524.  992.  745.  860.  638.  801.  777.  757.  722.  785.  321.  943.
  469.  541.  673.  716.  686. 1188.  969.  804.  728.  790.  741.  790.
  545.  591. 1091.  707.  475.  808.  750.  707.  556.  573.  562. 1057.
  750.  919.  511.  440.  638.  865.  588.  891.  731.  548.  437. 1015.
  878. 1017.  824.  993.  905.  600.  664.  890.  732.  696.  703.  348.
  690.  392. 1090. 1170.  799. 1191.  561.  840.  555.  918.  698.  687.
  735.  572.  926.  599.  469.  486.  973.  676.  817.  504.  388.  685.
  596.  878.  426.  862.  695.  542.  746.  694.  885.  524.  694.  935.
  715.  810.  978.  443.  318.  621.  805.  511. 1279.  832.  974.  737.
  857.  756.  595.  830.  526.  968.  974. 1279.  770.  980.  435.  881.
  858.  964.  454. 1068.  956.  874.  872.  623.  609.  623.  649.  620.
  850.  522.  999.  959.  593.  624.  488.  

In [11]:
# We can segregate the coefficients for constraints into a separate array
constr_coef = coef[1:,:-1]
print(constr_coef.shape)
print(constr_coef)

(5, 500)
[[702. 840. 513. ... 474. 707. 625.]
 [246. 994. 938. ... 734. 423. 691.]
 [799. 228.  10. ...  13. 231. 440.]
 [463. 654. 167. ... 574. 711. 678.]
 [252. 735. 586. ... 738. 947. 798.]]


In [12]:
# We can segregate the RHS of constraints into a separate array
constr_rhs = coef[1:,-1]
print(constr_rhs.shape)
print(constr_rhs)

(5,)
[173725. 190019. 186926. 192737. 183645.]


In [13]:
#setting upper and lower bound of variables
lower_bound = 0
upper_bound = 1

In [14]:
# set of column indices: since we have N variables, we can take column indices to be the list [0,1,2,...,N-1]
col_indices = np.arange(N)

In [15]:
#declare the decision variables in the model
model_lab7_ex2.x = Var(col_indices, domain=NonNegativeIntegers)

In [16]:
# set of row indices: since we have M contraints, we can take row indices to be the list [0,1,2,...,M-1]
row_indices = np.arange(M)

In [17]:
# create a ConstraintList to hold multiple constraints
model_lab7_ex2.constraints = ConstraintList()

In [18]:
for i in row_indices:
  model_lab7_ex2.constraints.add(summation(constr_coef[i],model_lab7_ex2.x) <= constr_rhs[i])

In [19]:
#add the upper bound and lower bounds for the variables
for j in col_indices:
  model_lab7_ex2.x[j].setlb(lower_bound)
  model_lab7_ex2.x[j].setub(upper_bound)

In [20]:
# add the model objective
model_lab7_ex2.objective = Objective(expr = summation(obj_coef,model_lab7_ex2.x), sense=maximize)

In [21]:
model_lab7_ex2.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :  500 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 15

In [22]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 159447 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [23]:
opt_cbc = SolverFactory('cbc')

In [24]:
result = opt_cbc.solve(model_lab7_ex2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time taken:', result.solver.time)
print(result.solver.statistics)

Solver status: ok
Solver termination condition: optimal
Solver time taken: 71.82229542732239

Branch and bound: 
  Number of bounded subproblems: 347044
  Number of created subproblems: 347044
Black box: 
  Number of iterations: 1407089



In [25]:
# display solution
print('\nObjective = ', model_lab7_ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab7_ex2.x[i].value)

print('\nConstraints')
model_lab7_ex2.constraints.display()


Objective =  295828.0

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0
x[ 12 ] =  0.0
x[ 13 ] =  1.0
x[ 14 ] =  1.0
x[ 15 ] =  0.0
x[ 16 ] =  1.0
x[ 17 ] =  0.0
x[ 18 ] =  1.0
x[ 19 ] =  0.0
x[ 20 ] =  1.0
x[ 21 ] =  1.0
x[ 22 ] =  0.0
x[ 23 ] =  0.0
x[ 24 ] =  0.0
x[ 25 ] =  1.0
x[ 26 ] =  1.0
x[ 27 ] =  1.0
x[ 28 ] =  0.0
x[ 29 ] =  0.0
x[ 30 ] =  0.0
x[ 31 ] =  1.0
x[ 32 ] =  0.0
x[ 33 ] =  1.0
x[ 34 ] =  1.0
x[ 35 ] =  1.0
x[ 36 ] =  1.0
x[ 37 ] =  1.0
x[ 38 ] =  0.0
x[ 39 ] =  1.0
x[ 40 ] =  1.0
x[ 41 ] =  1.0
x[ 42 ] =  1.0
x[ 43 ] =  1.0
x[ 44 ] =  1.0
x[ 45 ] =  0.0
x[ 46 ] =  1.0
x[ 47 ] =  1.0
x[ 48 ] =  1.0
x[ 49 ] =  1.0
x[ 50 ] =  1.0
x[ 51 ] =  1.0
x[ 52 ] =  1.0
x[ 53 ] =  1.0
x[ 54 ] =  1.0
x[ 55 ] =  0.0
x[ 56 ] =  0.0
x[ 57 ] =  0.0
x[ 58 ] =  1.0
x[ 59 ] =  1.0
x[ 60 ] =  1.0
x[ 61 ] =  1.0
x[ 62 ] =  0.0
x[ 63 ] =  0.0
x[ 64 ]

In [26]:
#changing domain of variables to continuous values
for i in col_indices:
    model_lab7_ex2.x[i].domain=NonNegativeReals

In [27]:
result = opt_cbc.solve(model_lab7_ex2)
print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time taken:', result.solver.time)
print(result.solver.statistics)

Solver status: ok
Solver termination condition: optimal
Solver time taken: 0.04897642135620117

Branch and bound: 
  Number of bounded subproblems: None
  Number of created subproblems: None
Black box: 
  Number of iterations: 18



In [28]:
# display solution
print('\nObjective = ', model_lab7_ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('x[',i,'] = ', model_lab7_ex2.x[i].value)

print('\nConstraints')
model_lab7_ex2.constraints.display()


Objective =  295896.37697236

Decision Variables
x[ 0 ] =  1.0
x[ 1 ] =  1.0
x[ 2 ] =  0.0
x[ 3 ] =  1.0
x[ 4 ] =  1.0
x[ 5 ] =  1.0
x[ 6 ] =  1.0
x[ 7 ] =  1.0
x[ 8 ] =  1.0
x[ 9 ] =  1.0
x[ 10 ] =  1.0
x[ 11 ] =  1.0
x[ 12 ] =  0.0
x[ 13 ] =  1.0
x[ 14 ] =  1.0
x[ 15 ] =  0.0
x[ 16 ] =  1.0
x[ 17 ] =  0.0
x[ 18 ] =  1.0
x[ 19 ] =  0.0
x[ 20 ] =  1.0
x[ 21 ] =  1.0
x[ 22 ] =  0.0
x[ 23 ] =  0.0
x[ 24 ] =  0.16942031
x[ 25 ] =  1.0
x[ 26 ] =  1.0
x[ 27 ] =  1.0
x[ 28 ] =  0.0
x[ 29 ] =  0.0
x[ 30 ] =  0.0
x[ 31 ] =  1.0
x[ 32 ] =  0.0
x[ 33 ] =  1.0
x[ 34 ] =  1.0
x[ 35 ] =  1.0
x[ 36 ] =  1.0
x[ 37 ] =  1.0
x[ 38 ] =  0.0
x[ 39 ] =  1.0
x[ 40 ] =  1.0
x[ 41 ] =  1.0
x[ 42 ] =  1.0
x[ 43 ] =  1.0
x[ 44 ] =  1.0
x[ 45 ] =  0.0
x[ 46 ] =  1.0
x[ 47 ] =  1.0
x[ 48 ] =  1.0
x[ 49 ] =  1.0
x[ 50 ] =  1.0
x[ 51 ] =  1.0
x[ 52 ] =  1.0
x[ 53 ] =  1.0
x[ 54 ] =  1.0
x[ 55 ] =  0.0
x[ 56 ] =  0.0
x[ 57 ] =  0.0
x[ 58 ] =  1.0
x[ 59 ] =  1.0
x[ 60 ] =  1.0
x[ 61 ] =  1.0
x[ 62 ] =  0.0
x[ 63 ] 

Answer 3) Optimal objective function value =  295828.0

Answer 4) Time taken to solve MILP = 71.82229542732239


Answer 5) Optimal objective function value = 295896.37697236

Answer 6) Time taken to solve LP = 0.04897642135620117
(which is much less than time taken to solve MILP which was 71.82229542732239)

Answer 7) Statistics in Branch and Bound section 

LP

Branch and bound: 

  Number of bounded subproblems: 347044

  Number of created subproblems: 347044

MILP

Branch and bound: 

  Number of bounded subproblems: None

  Number of created subproblems: None